In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv("avocado.csv")

In [4]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
Unnamed: 0      18249 non-null int64
Date            18249 non-null object
AveragePrice    18249 non-null float64
Total Volume    18249 non-null float64
4046            18249 non-null float64
4225            18249 non-null float64
4770            18249 non-null float64
Total Bags      18249 non-null float64
Small Bags      18249 non-null float64
Large Bags      18249 non-null float64
XLarge Bags     18249 non-null float64
type            18249 non-null object
year            18249 non-null int64
region          18249 non-null object
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


Поставим задачи по исследованию данного датасета:
1. Изучим взаимосвязь переменных
2. Проведем факторный анализ
3. Проведем двухфакторный дисперсионный анализ (влияние региона и даты на среднюю цену)
4. Построим линейную регрессию с целевой переменной - средней ценой
5. Оценим качество и статистическую значимость построенной линейной регрессии

Посмотрим на коэффициент корреляции Пирсона для наших признаков

In [6]:
df.corr()

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
Unnamed: 0,1.000000,-0.133008,0.014035,0.017628,0.019829,0.041752,-0.002219,0.000347,-0.009196,-0.011546,-0.171667
AveragePrice,-0.133008,1.000000,-0.192752,-0.208317,-0.172928,-0.179446,-0.177088,-0.174730,-0.172940,-0.117592,0.093197
Total Volume,0.014035,-0.192752,1.000000,0.977863,0.974181,0.872202,0.963047,0.967238,0.880640,0.747157,0.017193
4046,0.017628,-0.208317,0.977863,1.000000,0.926110,0.833389,0.920057,0.925280,0.838645,0.699377,0.003353
4225,0.019829,-0.172928,0.974181,0.926110,1.000000,0.887855,0.905787,0.916031,0.810015,0.688809,-0.009559
4770,0.041752,-0.179446,0.872202,0.833389,0.887855,1.000000,0.792314,0.802733,0.698471,0.679861,-0.036531
Total Bags,-0.002219,-0.177088,0.963047,0.920057,0.905787,0.792314,1.000000,0.994335,0.943009,0.804233,0.071552
Small Bags,0.000347,-0.174730,0.967238,0.925280,0.916031,0.802733,0.994335,1.000000,0.902589,0.806845,0.063915
Large Bags,-0.009196,-0.172940,0.880640,0.838645,0.810015,0.698471,0.943009,0.902589,1.000000,0.710858,0.087891
XLarge Bags,-0.011546,-0.117592,0.747157,0.699377,0.688809,0.679861,0.804233,0.806845,0.710858,1.000000,0.081033


Переменная TotalBags - это сумма переменных Small Bags, Large Bags и XLarge Bags. Поэтому коэффициент корреляции Пирсона для этих переменных близок к 1. Так же, мы видим что Total Volume достаточно сильно коррелирует с переменными 4046, 4225, 4770 и Total Bags. Таким образом, с помощью метода главных компонент, мы можем привести наши линейно зависимые факторы к меньшему числу линейно независимых. Для этого посмотрим на доли объясненной дисперсии.

In [7]:
data = df.copy()

In [8]:
data = data.drop('Unnamed: 0', axis=1)

In [9]:
data = data.drop('year', axis=1)

In [10]:
data = data.drop('AveragePrice', axis=1)

In [11]:
sigma2_total = data.var(axis=0).sum()

In [12]:
data.var(axis=0)/sigma2_total

Total Volume    0.719450
4046            0.096526
4225            0.087460
4770            0.000697
Total Bags      0.058673
Small Bags      0.033586
Large Bags      0.003590
XLarge Bags     0.000019
dtype: float64

Таким образом, данные 8 признаков можно свести к 5 с обяъсненной дисперсией 0,995695

In [13]:
Old_signs = np.array([data['Total Volume'], data['4046'], data['4225'], data['4770'], data['Total Bags'], data['Small Bags'],\
                     data['Large Bags'], data['XLarge Bags']])

In [14]:
Old_signs.shape

(8, 18249)

In [15]:
means = np.array([Old_signs.T.mean(axis=0)])
ones = np.ones((1,18249))
means_matrix = means.T.dot(ones)

In [17]:
covariance_matrix = np.cov(Old_signs, ddof=1)

In [18]:
eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

In [19]:
eig_values

array([1.63121576e+13, 1.41871133e+11, 1.09809106e+11, 1.10044342e+10,
       2.95317374e+09, 1.20818233e+08, 5.11930066e+02, 1.13534742e-03])

In [20]:
Old_signs = Old_signs - means_matrix

Возьмем 5 первых собственных значений и соответствующих им собственных векторов, составим преобразующую матрицу из собственных векторов и найдем тем самым матрицу новых признаков.

In [21]:
T_matrix = np.array([eig_vectors[x] for x in range(0,5)]).T
T_matrix

array([[ 8.55061262e-01,  3.06232053e-01,  2.90133294e-01,
         2.31337735e-02,  2.35562170e-01],
       [-5.80766605e-02, -1.21854716e-01, -5.45854077e-01,
        -5.59563663e-02,  6.65583895e-01],
       [ 1.29097713e-03, -7.87892857e-01,  5.58407431e-01,
         3.03951034e-02,  2.00378007e-01],
       [ 6.70978003e-02, -5.95030929e-02, -1.30753899e-02,
        -1.41059136e-04,  1.39827233e-01],
       [ 2.20170751e-01, -2.34365653e-01, -3.06853612e-01,
         8.76957080e-01, -1.15562926e-01],
       [-4.45884279e-02,  4.00229928e-02,  3.44668836e-02,
         1.25787305e-01, -2.45034438e-01],
       [ 4.58835777e-01, -4.58833833e-01, -4.58834467e-01,
        -4.58839477e-01, -3.44105370e-01],
       [ 4.85863645e-07, -4.84513779e-07, -4.86962395e-07,
        -4.84703049e-07, -5.00000362e-01]])

In [22]:
New_signs = Old_signs.T.dot(T_matrix)
New_signs.shape

(18249, 5)

In [23]:
New_data = pd.DataFrame(New_signs)
New_data.head()

,0,1,2,3,4
0,-725305.827960,57823.418985,-113127.787277,-208640.164764,-341553.980322
1,-733155.706794,62575.469205,-121349.330873,-208326.337058,-346257.533006
2,-679148.209645,31388.114061,-66646.963774,-206274.149156,-317829.535019
3,-713156.505730,49064.470188,-98348.819333,-210688.890624,-333463.816216
4,-736985.055227,62592.932811,-122200.389737,-211843.698369,-345953.740224


Теперь мы хотим изучить влияние даты и региона на переменную AvaragePrice. Для этого зафиксируем поле type. Таким образом мы получим для каждой пары дата - регион единственное значение целевой переменной.

In [24]:
data_2 = df.copy()

In [25]:
data_2.loc[data_2['type'] == 'conventional']['Date'].value_counts().count()

169

In [26]:
data_2.loc[data_2['type'] == 'conventional']['region'].value_counts().count()

54

Таким образом мы имеем 169 уровней для фактора даты и 54 уровня для фактора региона. Составим матрицу значений для этих двух факторов.

In [27]:
def values_matrix(data):
    res_matrix = []
    for row in data['region'].value_counts().keys():
        x = []
        for column in data['Date'].value_counts().keys():
            x = np.append(x, data.loc[data['Date'] == column].loc[data['region'] == row]['AveragePrice'])
        if np.any(res_matrix):
            try:
                res_matrix = np.append(res_matrix, np.array([x]), axis=0)
            except Exception as e:
                print(e)
                break
        else:
            res_matrix = np.array([x])
    return res_matrix

In [28]:
M = values_matrix(data_2.loc[data_2['type'] == 'conventional'])
M

array([[0.95, 0.9 , 1.13, ..., 1.31, 1.06, 1.  ],
       [1.15, 0.9 , 1.31, ..., 1.32, 1.03, 0.92],
       [1.29, 1.12, 1.48, ..., 1.4 , 1.2 , 1.3 ],
       ...,
       [1.55, 1.14, 1.75, ..., 1.7 , 1.45, 1.29],
       [0.73, 0.7 , 0.9 , ..., 0.94, 0.72, 0.78],
       [1.33, 1.17, 1.36, ..., 1.34, 1.23, 1.02]])

In [29]:
M.shape

(54, 169)

In [30]:
def square_w(matrix):
    res = 0
    for i, elements in enumerate(matrix):
        for j, element in enumerate(elements):
            res += (element - matrix.mean(axis=1)[i] - matrix.mean(axis=0)[j] + matrix.mean())**2
    return res

In [31]:
m = M.shape[1]
k = M.shape[0]
m, k

(169, 54)

In [32]:
S2_date = k*((M.mean(axis=0)-M.mean())**2).sum()
S2_region = m*((M.mean(axis=1)-M.mean())**2).sum()
S2_w = square_w(M)

In [33]:
sigma2_date = S2_date/(m-1)
sigma2_region = S2_region/(k-1)
sigma2_w = S2_w/((k-1)*(m-1))

Статистики для проверки:

In [34]:
F_date = sigma2_date/sigma2_w
F_region = sigma2_region/sigma2_w
F_date, F_region

(103.99290096725106, 296.8705115087351)

Теперь найдем критическое значение с помощью F-критерия фишера:

In [35]:
from scipy import stats

In [36]:
alpha = 0.05
f_crit_date = stats.f.ppf(1-alpha, m-1, m*k-m)
f_crit_region = stats.f.ppf(1-alpha, k-1, m*k-k)
f_crit_date, f_crit_region

(1.1881711421704246, 1.340974298402173)

Значения статистик оказалось больше чем критические значения, а значит два этих фактора влияют на переменную AveragePrice

К сожалнию для type = organic нам не хватает значений AveragePrice для некоторых пар даты-региона.

Теперь, построим новый датафрейм с нашими данными и построим линейную регрессию на полученных данных с целевой переменной AveragePrice

In [37]:
l = New_data.copy()

In [38]:
data_3 = df.copy()

In [39]:
data_3 = data_3.drop('Unnamed: 0', axis=1)

In [40]:
data_3 = data_3.drop('Total Volume', axis=1)

In [41]:
data_3 = data_3.drop('4046', axis=1)

In [42]:
data_3 = data_3.drop('4225', axis=1)

In [43]:
data_3 = data_3.drop('4770', axis=1)

In [44]:
data_3 = data_3.drop('Total Bags', axis=1)

In [45]:
data_3 = data_3.drop('Small Bags', axis=1)

In [46]:
data_3 = data_3.drop('Large Bags', axis=1)

In [47]:
data_3 = data_3.drop('XLarge Bags', axis=1)

In [48]:
data_3 = data_3.drop('year', axis=1)

In [49]:
data_3 = data_3.merge(l,left_index=True, right_index=True)

In [50]:
data_3.head()

,Date,AveragePrice,type,region,0,1,2,3,4
0,2015-12-27,1.33,conventional,Albany,-725305.827960,57823.418985,-113127.787277,-208640.164764,-341553.980322
1,2015-12-20,1.35,conventional,Albany,-733155.706794,62575.469205,-121349.330873,-208326.337058,-346257.533006
2,2015-12-13,0.93,conventional,Albany,-679148.209645,31388.114061,-66646.963774,-206274.149156,-317829.535019
3,2015-12-06,1.08,conventional,Albany,-713156.505730,49064.470188,-98348.819333,-210688.890624,-333463.816216
4,2015-11-29,1.28,conventional,Albany,-736985.055227,62592.932811,-122200.389737,-211843.698369,-345953.740224


In [51]:
data_3 = pd.get_dummies(data_3)

Для того чтобы получить хорошую линейную регрессию с использованием dummie-переменных, необходимо нормировать данные. Воспользуемся стандартным скейлером из библиотеки Sklearn

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [53]:
y = data_3['AveragePrice']
x = data_3.drop('AveragePrice', axis=1)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=50)

In [55]:
scaler = StandardScaler()

In [56]:
X_train_scaled = scaler.fit_transform(X_train)

In [57]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

In [58]:
X_train_scaled.head()

,0,1,2,3,4,Date_2015-01-04,Date_2015-01-11,Date_2015-01-18,Date_2015-01-25,Date_2015-02-01,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
0,0.639956,-0.222136,-0.411671,0.629678,0.503603,-0.079386,-0.078382,-0.080379,-0.075289,-0.077874,...,-0.137492,-0.135708,7.121559,-0.137492,-0.138082,-0.140997,-0.138962,-0.136007,-0.138962,-0.140129
1,-0.139147,-0.094366,0.068779,-0.127161,-0.166077,-0.079386,-0.078382,-0.080379,-0.075289,-0.077874,...,-0.137492,-0.135708,-0.140419,-0.137492,-0.138082,-0.140997,-0.138962,-0.136007,-0.138962,-0.140129
2,-0.128288,0.088185,0.060418,-0.158678,-0.158884,-0.079386,-0.078382,-0.080379,-0.075289,-0.077874,...,-0.137492,-0.135708,-0.140419,-0.137492,-0.138082,-0.140997,-0.138962,-0.136007,-0.138962,-0.140129
3,-0.238576,0.237767,-0.230828,-0.228986,-0.233170,12.596627,-0.078382,-0.080379,-0.075289,-0.077874,...,-0.137492,-0.135708,7.121559,-0.137492,-0.138082,-0.140997,-0.138962,-0.136007,-0.138962,-0.140129
4,-0.175276,-0.027612,0.019609,-0.181130,-0.181566,-0.079386,-0.078382,-0.080379,-0.075289,-0.077874,...,-0.137492,-0.135708,-0.140419,-0.137492,-0.138082,-0.140997,-0.138962,-0.136007,-0.138962,-0.140129


In [59]:
model = LR()

In [60]:
model.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Можно увидеть коэффициенты нашей модели

In [61]:
model.coef_

array([ 6.30045841e-01,  1.71877303e-01,  1.32963919e-01, -2.29604163e-01,
       -3.80103785e-01, -8.01525826e-03, -2.55785594e-03,  8.54817148e-04,
       -2.43215437e-04, -1.17756213e-02, -7.76781884e-03, -1.88847049e-03,
       -3.97359208e-03, -8.33015772e-03, -3.72681378e-03, -1.28364309e-03,
       -5.86898618e-03, -6.75414977e-04,  1.17397770e-03, -3.61085520e-03,
       -3.57460032e-03, -1.58701661e-03, -9.04254849e-03, -6.52986822e-03,
       -4.27564026e-03, -3.51886656e-03, -3.22141822e-03, -3.32983717e-03,
       -2.59472289e-04,  8.97602375e-04, -1.59079886e-03,  1.52153391e-03,
        6.78475848e-04, -3.58504028e-04,  1.10598892e-03,  4.86723782e-03,
        2.34060220e-03,  3.86167757e-03,  2.54554774e-03,  1.22865726e-03,
        5.78540950e-04,  4.15472352e-03,  4.22862097e-03,  3.59922212e-03,
        5.55193651e-04, -2.79809588e-03, -3.17258144e-03, -2.23529633e-03,
       -7.30334049e-03, -7.01863366e-03, -6.12552945e-03, -6.27714051e-03,
       -4.34147444e-03, -

In [62]:
X_test_scaled = scaler.transform(X_test)

In [63]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [64]:
y_pred = model.predict(X_test_scaled)

In [65]:
r2 = r2_score(y_test, y_pred)
r2

0.6834097534892042

Так же, можно найти среднюю ошибку аппроксимации.

In [66]:
from sklearn.metrics import mean_absolute_error

In [67]:
A = mean_absolute_error(y_test, y_pred)/y_test.abs().sum()

In [68]:
A

2.162147317771e-05

Не смотря на небольшой R2, средняя ошибка аппроксимации невелика.

Оценим теперь статистическую значимость построенной модели с помощью F-критерия Фишера.

In [69]:
k = len(X_train.columns)
n = len(X_train['Date_2015-01-04'])
F = (r2/k)/((1-r2)/(n-k-1))
F

117.72189051921767

In [70]:
alpha = 0.05
f_crit = stats.f.ppf(1-alpha, k, n-k-1)
f_crit

1.1599315787126918

Таким образом построенное уравнение линейной регрессии статистически значимо.